In [1]:
## Bruno Vieira Ribeiro - 2021

In [1]:
%config Completer.use_jedi = False        # For autocomplete to work!

<H1 align="center">  🧸 Project: Toys 🧸 </H1>

---

# Data integrity

In [2]:
import pandas as pd
import sqlite3 as sql

# Connecting to the database

In [3]:
con = sql.connect("kohls_toys.db")
cur = con.cursor()

## Listing tables

In [4]:
# Listing all tables
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()
print(tables)
print(50*'=')
print(f'Total of {len(tables)} tables in database.')

[('girlstoys',), ('boystoys',)]
Total of 2 tables in database.


Number of rows (different toys) in each table:

In [5]:
query = """
SELECT
    Count(*)
FROM
    girlstoys;
"""
cur.execute(query)
print(f'Number of girls toys: {cur.fetchall()[0][0]}')

Number of girls toys: 4475


In [6]:
query = """
SELECT
    Count(*)
FROM
    boystoys;
"""
cur.execute(query)
print(f'Number of boys toys: {cur.fetchall()[0][0]}')

Number of boys toys: 4210


# Checking for duplicates

## Girls toys

Before doing any analysis we can start by checking if there are duplicate toy entries in our tables. We can do this with a `GROUP BY` statement filtering all entries `HAVING` a `COUNT` greater than 1:

In [7]:
query = """
SELECT
    name,
    description,
    COUNT(*) AS cnt
FROM
    girlstoys
GROUP BY
    name,
    description
HAVING COUNT(*) > 1;
"""
check_df = pd.read_sql(query, con)
check_df

,name,description,cnt
0,American Girl Little Ladybug Brush,Use this American Girl Little Ladybug Brush to...,2
1,Barbie® Face Mask Spa Day Playset,Barbie doll knows the way to be one's best is ...,2
2,Barbie® Slap Bracelet Activity Set,Your little artist will love creating all sort...,2
3,Barbie® Stroll 'n' Play Pups Doll and Accessories,Animal lovers can help Barbie doll take her pu...,2
4,Candy Land 65th Anniversary by Winning Moves,Candy Land 65th Anniversary by Winning Moves f...,2
...,...,...,...
63,Think Box Build & Create Robotics STEM Set for...,Your little engineer will love learning how to...,2
64,University Games 3D Crystal Puzzle - Disney's ...,Bust boredom by putting together this Universi...,2
65,University Games 3D Crystal Puzzle - Dragon 56...,Bust boredom by putting together this Universi...,3
66,Winfun Balls 'N Shapes Musical Table,Keep your busy bee stimulated and occupied wit...,2


So, there are 63 duplicate toys in this table. Let's just delete them from the table before doing further analysis. To delete them we can use the `ROWID`. The idea is to keep only the duplicated (sometimes triplicated!) products with the lowest `ROWID`. We do this by deleting all entries where the `ROWID` is **not in** the `MIN` `ROWID` from our grouped data:

In [8]:
delete_duplicates = """
DELETE FROM girlstoys
WHERE ROWID NOT IN(
    SELECT
        MIN(ROWID)
    FROM
        girlstoys
    GROUP BY
        name,
        description
    )
"""
cur.execute(delete_duplicates)

In [9]:
con.commit()

Sanity check: let's check for duplictaes again.

In [10]:
query = """
SELECT
    name,
    description,
    COUNT(*) AS cnt
FROM
    girlstoys
GROUP BY
    name,
    description
HAVING COUNT(*) > 1;
"""
check_df = pd.read_sql(query, con)
check_df

,name,description,cnt


Great, no duplicates!

## Boys toys

We can reuse all the code above to find and delete duplicates from the `boystoys` table.

In [11]:
query = """
SELECT
    name,
    description,
    COUNT(*) AS cnt
FROM
    boystoys
GROUP BY
    name,
    description
HAVING COUNT(*) > 1;
"""
check_df = pd.read_sql(query, con)
check_df

,name,description,cnt
0,Disney Princess Belle and Rapunzel's Royal Sta...,Kick-start fun role-play with this LEGO Disney...,2
1,Dynacraft PJ Masks 6 Volt Ride On Car,Calling all PJ Masks fans to this Dynacraft ri...,2
2,Educational Insights Design & Drill My First W...,See what your little builder can create with t...,2
3,Hey! Play! Zigzag Ride-On Vehicle,Kids will love to wiggle around in this Hey! P...,2
4,KidKraft Airport Express Train Set & Table,Your little locomotive fan will have a blast p...,2
5,LEGO City Fire Helicopter Response 60248 Build...,An awesome building set for kids who love acti...,2
6,Learning Resources Pretend & Play Tool Set,Bring out the carpenter in your little builder...,2
7,Lil' Rider FX 3-Wheel Bike Ride-On,"With a working headlight and sound effects, th...",2
8,Lionel The Polar Express 50ft Flatcar,Update your Christmas track with this Polar Ex...,2
9,Melissa & Doug Giant Lifelike Plush Stork,This lifelike plush stork from Melissa & Doug ...,2


There are 26 duplicates here. Let's delete them:

In [12]:
delete_duplicates = """
DELETE FROM boystoys
WHERE ROWID NOT IN(
    SELECT
        MIN(ROWID)
    FROM
        boystoys
    GROUP BY
        name,
        description
    )
"""
cur.execute(delete_duplicates)

In [13]:
con.commit()

Sanity check:

In [14]:
query = """
SELECT
    name,
    description,
    COUNT(*) AS cnt
FROM
    boystoys
GROUP BY
    name,
    description
HAVING COUNT(*) > 1;
"""
check_df = pd.read_sql(query, con)
check_df

,name,description,cnt


Awesome, let's move on!

## Reading tables to different dataframes

In [15]:
girls_df = pd.read_sql('SELECT * FROM girlstoys', con)

In [16]:
girls_df.head()

,name,price,url,description
0,Barbie® 3-in-1 DreamCamper Vehicle and Accesso...,89.99,https://www.kohls.com/product/prd-4469399/barb...,When kids hit the open road in the Barbie 3-in...
1,Barbie® Dreamhouse Playset,199.99,https://www.kohls.com/product/prd-5115185/barb...,"Measuring 43 inches tall and 41 inches wide, t..."
2,Step2 Modern Cook Play Kitchen Set,79.99,https://www.kohls.com/product/prd-1819866/step...,Fire up your little one's love for cooking wit...
3,Barbie® Dreamhouse Playset,199.99,https://www.kohls.com/product/prd-5184120/barb...,The possibilities are endless for your creativ...
4,Hot Wheels Race Car & Track Play Table by Step2,119.99,https://www.kohls.com/product/prd-3414834/hot-...,Your little speedster will love all the racing...


In [17]:
boys_df = pd.read_sql('SELECT * FROM boystoys', con)

In [18]:
boys_df.head()

,name,price,url,description
0,Step2 Modern Cook Play Kitchen Set,79.99,https://www.kohls.com/product/prd-1819866/step...,Fire up your little one's love for cooking wit...
1,CoComelon Musical Yellow School Bus Pop Up Tent,29.99,https://www.kohls.com/product/prd-5190142/coco...,Kids will have a great time when they play and...
2,Hot Wheels Race Car & Track Play Table by Step2,119.99,https://www.kohls.com/product/prd-3414834/hot-...,Your little speedster will love all the racing...
3,LEGO Harry Potter Hogwarts Chamber of Secrets ...,129.99,https://www.kohls.com/product/prd-5167485/lego...,"The large, modular LEGO Harry Potter Hogwarts ..."
4,Hot Wheels City Ultimate Garage,99.99,https://www.kohls.com/product/prd-4429150/hot-...,"New for 2020, it's the ULTIMATE challenge in t..."


## Checking integrity of `price` column

All entries in our price columns should be float numbers. As a integrity check, we can try to cast them (as they are currently TEXT) as `FLOAT` and see if any rows fail the cast.

Let's start with girls toys:

In [20]:
query = """
SELECT
    *
FROM
    girlstoys
WHERE
    CAST(price AS FLOAT) IS NOT price;
"""
check_df = pd.read_sql(query, con)
check_df

,name,price,url,description
0,KidKraft Teepee Play Tent,99.99 - $112.99,https://www.kohls.com/product/prd-2034244/kidk...,Featuring sturdy bamboo and soft cotton canvas...
1,Delta Children MySize Double-Sided Storage Easel,62.09 - $69.99,https://www.kohls.com/product/prd-4584636/delt...,This two-sided easel features a chalkboard sid...
2,Melissa & Doug Diner Collection,18.99 - $248.99,https://www.kohls.com/product/prd-c2228951/mel...,No info
3,Little Tikes 4-in-1 Sports Edition Trike,112.99 - $133.99,https://www.kohls.com/product/prd-1882301/litt...,"Designed with both parent and child in mind, t..."
4,Melissa & Doug Let's Play House Collection,16.99 - $215.99,https://www.kohls.com/product/prd-c1441954/mel...,No info


In [21]:
len(check_df)

5

Ok, so there are five rows with a price range instead of a single price in the `girlstoys` table. Let's inspect the `boystoys` table:

In [22]:
query = """
SELECT
    *
FROM
    boystoys
WHERE
    CAST(price AS FLOAT) IS NOT price;
"""
check_df = pd.read_sql(query, con)
check_df

,name,price,url,description
0,KidKraft Teepee Play Tent,99.99 - $112.99,https://www.kohls.com/product/prd-2034244/kidk...,Featuring sturdy bamboo and soft cotton canvas...
1,Delta Children MySize Double-Sided Storage Easel,62.09 - $69.99,https://www.kohls.com/product/prd-4584636/delt...,This two-sided easel features a chalkboard sid...
2,Melissa & Doug Diner Collection,18.99 - $248.99,https://www.kohls.com/product/prd-c2228951/mel...,No info
3,Little Tikes 4-in-1 Sports Edition Trike,112.99 - $133.99,https://www.kohls.com/product/prd-1882301/litt...,"Designed with both parent and child in mind, t..."
4,Melissa & Doug Let's Play House Collection,16.99 - $215.99,https://www.kohls.com/product/prd-c1441954/mel...,No info


In [23]:
len(check_df)

5

The same five rows appear in this table. Let's modify it to cast all entries as numeric.

### Making modifications

All the non-numeric values of price are a range represented by values separated by an "-". So, we can use the `SUBSTR` function along with `INSTR` to get the lowest value in the range (a bit arbitrary because I want to see the lowest possible price for each toy, but a similar idea could be implemented for the highest).

The goal is to find, in each of these incorrect inputs, the substring starting from the first character and up to the index of the "-" character (minus 2: to consider it and the white space just before it).

In SQLite, the function `SUBSTR` takes 3 arguments:

`SUBSTR(X,Y,Z)`
* The x field represents the string input to be sliced, 
* the y and z field represents the starting point and ending point respectively using an index.

The `INSTR` function takes 2 arguments:

`INSTR(string, substring)`
* string is the source string that INSTR function searches for the substring

* substring is the substring that is used to search.

Combining these two we can filter the substring starting at index 1 and going up to the index where the "-" is (minus 2).

In [24]:
query = """
SELECT
    price,
    SUBSTR(price, 1, INSTR(price, '-') - 2) AS correct_price
FROM
    girlstoys
WHERE
    CAST(price AS FLOAT) IS NOT price;
"""
check_df = pd.read_sql(query, con)
check_df

,price,correct_price
0,99.99 - $112.99,99.99
1,62.09 - $69.99,62.09
2,18.99 - $248.99,18.99
3,112.99 - $133.99,112.99
4,16.99 - $215.99,16.99


### Updating the tables

Now that we know how to make the desired modifications, we can update the `price` column in each table. We do this using a **searched CASE expression** combined with an **UPDATE**. The basic syntax of this expression is as follows:
```sql
CASE
     WHEN bool_expression_1 THEN result_1
     WHEN bool_expression_2 THEN result_2
     [ ELSE result_else ] 
END
```

From the SQLite documentation:
> The searched CASE expression evaluates the Boolean expressions in the sequence specified and return the corresponding result if the expression evaluates to true.

Using this, we can update the `girlstoys` table first with the query bellow:

In [25]:
update_date = """
UPDATE girlstoys
    SET price =
        CASE
        WHEN
            CAST(price AS FLOAT) IS NOT price THEN SUBSTR(price, 1, INSTR(price, '-') - 2)
        ELSE 
            price
        END
"""
cur.execute(update_date)

In [26]:
con.commit()

To check if it worked we can run the same query we did the first time we were checking for integrity.

In [27]:
query = """
SELECT
    *
FROM
    girlstoys
WHERE
    CAST(price AS FLOAT) IS NOT price;
"""
check_df = pd.read_sql(query, con)
check_df

,name,price,url,description


Great! Seems ok now. We can repeat the process for the `boystoys` table:

In [28]:
update_date = """
UPDATE boystoys
    SET price =
        CASE
        WHEN
            CAST(price AS FLOAT) IS NOT price THEN SUBSTR(price, 1, INSTR(price, '-') - 2)
        ELSE 
            price
        END
"""
cur.execute(update_date)

In [29]:
con.commit()

Our quick sanity check:

In [30]:
query = """
SELECT
    *
FROM
    boystoys
WHERE
    CAST(price AS FLOAT) IS NOT price;
"""
check_df = pd.read_sql(query, con)
check_df

,name,price,url,description


Awesome! Our `price` columns is ready to be analysed safely! We can close the connection to the database.

In [31]:
con.close()